# Nibabel

Nibabel is a low-level Python library that gives access to a variety of imaging formats, with a particular focus on providing a common interface to the various **volumetric** formats produced by scanners and used in common neuroimaging toolkits.

 - NIfTI-1
 - NIfTI-2
 - SPM Analyze
 - FreeSurfer .mgh/.mgz files
 - Philips PAR/REC
 - Siemens ECAT
 - DICOM (limited support)

It also supports **surface** file formats

 - GIFTI
 - FreeSurfer surfaces, labels and annotations

**Connectivity**

 - CIFTI-2

**Tractocgraphy**

 - TrackViz .trk files

And a number of related formats.

**Note:** Almost all of these can be loaded through the `nibabel.load` interface.

## Setup

In [ ]:
# Image settings
import pylab as plt
%matplotlib inline

import numpy as np
import nibabel as nb

## Loading and inspecting images in `nibabel`

In [ ]:
# Load a functional image of subject 01
img = nb.load('/data/ds000114/sub-02/ses-test/func/sub-02_ses-test_task-fingerfootlips_bold.nii.gz')

In [ ]:
# Let's look at the header of this file
print(img)

This data-affine-header structure is common to volumetric formats in nibabel, though the details of the header will vary from format to format.

### Access specific parameters

If you're interested in specific parameters, you can access them very easily, as the following examples show.

In [ ]:
data = img.get_data()
data.shape

In [ ]:
affine = img.affine
affine

In [ ]:
header = img.header['pixdim']
header

Note that in the `'pixdim'` above contains the voxel resolution  (`3.125., 3.125, 4.2`), as well as the TR (`2.`).

### Data

The data is a simple numpy array. It has a shape, it can be sliced and generally manipulated as you would any array.

In [ ]:
plt.imshow(data[:, :, data.shape[2] // 2, 0].T, cmap='Greys_r')
print(data.shape)

In [ ]:
t1 = nb.load('/data/ds000114/sub-02/ses-test/anat/sub-02_ses-test_T1w.nii.gz')
data = t1.get_data()
plt.imshow(data[:, :, data.shape[2] // 2].T, cmap='Greys_r')
print(data.shape)

## Creating and saving images

Suppose we want to save space by rescaling our image to a smaller datatype, such as an unsigned byte. To do this, we first need to take the data, change it's datatype and save this new data in a new NIfTI image with the same header and affine as the original image.

In [ ]:
# First, we need to load the image and get the data
img = nb.load('/data/ds000114/sub-02/ses-test/func/sub-02_ses-test_task-fingerfootlips_bold.nii.gz')
data = img.get_data()

In [ ]:
# Now we force the values to be between 0 and 255
# and change the datatype to unsigned 8-bit
rescaled = ((data - data.min()) * 255. / (data.max() - data.min())).astype(np.uint8)

In [ ]:
# Now we can save the changed data into a new NIfTI file
new_img = nb.Nifti1Image(rescaled, affine=img.affine, header=img.header)
nb.save(new_img, '/tmp/rescaled_image.nii.gz')

# Nilearn

[Nilearn](http://nilearn.github.io/index.html) labels itself as: *A Python module for fast and easy statistical learning on NeuroImaging data. It leverages the scikit-learn Python toolbox for multivariate statistics with applications such as predictive modelling, classification, decoding, or connectivity analysis.*

But it's much more than that. It is also an excellent library to **manipulate** (e.g. resample images, smooth images, ROI extraction, etc.) and **visulaize** your neuroimages.

So let's visit all three of those domains:

1. Image manipulation
2. Image visualization

## Setup

In [ ]:
# Image settings
from nilearn import plotting
import pylab as plt
%matplotlib inline

import numpy as np

Throughout this tutorial we will be using the anatomical and functional image of subject 1. So let's load them already here that we have a quicker access later on:

In [ ]:
from nilearn import image as nli
t1 = nli.load_img('/data/ds000114/sub-02/ses-test/anat/sub-02_ses-test_T1w.nii.gz')
bold = nli.load_img('/data/ds000114/sub-02/ses-test/func/sub-02_ses-test_task-fingerfootlips_bold.nii.gz')

Because the bold image didn't reach steady-state at the beginning of the image, let's cut of the first 5 volumes, to be sure:

In [ ]:
bold = bold.slicer[..., 5:]

## 1. Image manipulation with `nilearn`

### Let's create a mean image

If you use nibabel to compute the mean image, you first need to load the img, get the data and then compute the mean thereof. With nilearn you can do all this in just one line with `mean image`.

In [ ]:
from nilearn import image as nli

In [ ]:
img = nli.mean_img(bold)

In [ ]:
img.orthoview()

Perfect! What else can we do with the `image` module?  
Let's see...

### Resample image to a template
Using `resample_to_img`, we can resample one image to have the same dimensions as another one. For example, let's resample an anatomical T1 image to the computed mean image above.

In [ ]:
mean = nli.mean_img(bold)
print([mean.shape, t1.shape])

Let's resample the t1 image to the mean image.

In [ ]:
resampled_t1 = nli.resample_to_img(t1, mean)
resampled_t1.shape

The image size of the resampled t1 image seems to be right. But what does it look like?

In [ ]:
from nilearn import plotting
plotting.plot_anat(t1, title='original t1', display_mode='z', dim=-1,
                   cut_coords=[-20, -10, 0, 10, 20, 30])
plotting.plot_anat(resampled_t1, title='resampled t1', display_mode='z', dim=-1,
                   cut_coords=[-20, -10, 0, 10, 20, 30])

### Smooth an image
Using `smooth_img`, we can very quickly smooth any kind of MRI image. Let's for example take the mean image from above and smooth it with different FWHM values.

In [ ]:
for fwhm in range(1, 12, 5):
    smoothed_img = nli.smooth_img(mean, fwhm)
    plotting.plot_epi(smoothed_img, title="Smoothing %imm" % fwhm,
                     display_mode='ortho', cmap='magma')

### Mask an image and extract an average signal of a region

Thanks to nibabel and nilearn you can consider your images just a special kind of a numpy array. Which means that you have all the liberties that you are used to.

For example, let's take a functional image, (1) create the mean image thereof, than we (2) threshold it to only keep the voxels that have a value that is higher than 95% of all voxels. Of this thresholded image, we only (3) keep those regions that are bigger than 1000mm^3. And finally, we (4) binarize those regions to create a mask image.

So first, we load again a functional image and compute the mean thereof.

In [ ]:
mean = nli.mean_img(bold)

Use `threshold_img` to only keep voxels that have a value that is higher than 95% of all voxels.

In [ ]:
thr = nli.threshold_img(mean, threshold='95%')

In [ ]:
thr.orthoview()

Now, let's only keep those voxels that are in regions/clusters that are bigger than 1000mm^3.

In [ ]:
voxel_size = np.prod(thr.header['pixdim'][1:4])  # Size of 1 voxel in mm^3
voxel_size

Let's create the mask that only keeps those big clusters.

In [ ]:
from nilearn.regions import connected_regions
cluster = connected_regions(thr, min_region_size=1000. / voxel_size, smoothing_fwhm=1)[0]

And finally, let's binarize this cluster file to create a mask.

In [ ]:
mask = nli.math_img('np.mean(img,axis=3) > 0', img=cluster)

Now let us investigate this mask by visualizing it on the subject specific anatomy:

In [ ]:
from nilearn.plotting import plot_roi
plotting.plot_roi(mask, bg_img=t1, display_mode='ortho', dim=-.5, cmap='magma_r');

Next step is now to take this mask, apply it to the original functional image and extract the mean of the temporal signal.

In [ ]:
# Apply mask to original functional image
from nilearn.masking import apply_mask

all_timecourses = apply_mask(bold, mask)
all_timecourses.shape

**Note:** You can bring the timecourses (or masked data) back into the original 3D/4D space with `unmask`:

In [ ]:
from nilearn.masking import unmask
img_timecourse = unmask(all_timecourses, mask)

Compute mean trace of all extractet timecourses and plot the mean signal.

In [ ]:
mean_timecourse = all_timecourses.mean(axis=1)
plt.plot(mean_timecourse)

## 2. Image visualization with `nilearn`

Above, we've already seen some ways on how to visualize brain images with `nilearn`. And there are many more. To keep this notebook short, we will only take a look at some of them. For a complete list, see [nilearn's plotting section](http://nilearn.github.io/plotting/index.html).

**Note:** In most of the `nilearn`'s plotting functions, you can specify the value `output_file=example.png'`, to save the figure directly to a file.